In [ ]:
import os
import re
import json
from collections import defaultdict

"""
find data/block-textures/block -name "*.png" | sort > block_textures.txt
find data/block-textures/block -name "*.mcmeta" | sort > animated_textures.txt
"""
# Read texture files
with open('block_textures.txt', 'r') as f:
    texture_files = [line.strip() for line in f.readlines()]

# Read animated texture files
with open('animated_textures.txt', 'r') as f:
    animated_files = [line.strip() for line in f.readlines()]

# Convert animated files to corresponding png files
animated_pngs = [animated.replace('.mcmeta', '') for animated in animated_files]

# Filter out animated textures
non_animated_textures = [texture for texture in texture_files if texture not in animated_pngs]

# Group by block prefix
block_textures = defaultdict(list)

for texture in non_animated_textures:
    # Extract filename without path
    filename = os.path.basename(texture)
    
    # Try to extract block name from filename
    # This regex looks for patterns like "block_name_additional_info.png"
    match = re.match(
        r'([a-z_]+?)(?:_(?:top|bottom|side|front|back|end|on|off|lit|'
        r'stage\d+|\d+|plant))?\.png$', 
        filename
    )
    
    if match:
        block_name = match.group(1)
        block_textures[block_name].append(filename)
    else:
        # For files that don't match the pattern, use the full name without extension
        block_name = os.path.splitext(filename)[0]
        block_textures[block_name].append(filename)

# Print results
print(f"Found {len(texture_files)} total texture files")
print(f"Found {len(animated_pngs)} animated texture files")
print(f"Found {len(non_animated_textures)} non-animated texture files")
print(f"Grouped into {len(block_textures)} block types\n")

# Print the blocks and their textures
for block, textures in sorted(block_textures.items()):
    print(f"{block}: {len(textures)} textures")
    for texture in sorted(textures):
        print(f"  - {texture}")
    print()

# Save results to JSON file
result = {
    "stats": {
        "total_textures": len(texture_files),
        "animated_textures": len(animated_pngs),
        "non_animated_textures": len(non_animated_textures),
        "block_types": len(block_textures)
    },
    "blocks": {block: sorted(textures) for block, textures in block_textures.items()}
}

with open('minecraft_block_textures.json', 'w') as f:
    json.dump(result, f, indent=2) 